In [ ]:
# --- OPERATIONAL BASELINE: MODEL OUTPUT STATISTICS (MOS) ---
# This script creates a strong, non-deep-learning baseline to compare against.
# It uses a classic statistical downscaling technique: a separate multiple linear
# regression model is trained for each pixel in the high-resolution grid.
# This serves as the final, crucial comparison to prove the value of the deep
# learning approach over traditional statistical methods.

import numpy as np
from pathlib import Path
from tqdm import tqdm
import joblib
from sklearn.linear_model import LinearRegression
from scipy.ndimage import zoom
import warnings

# --- Imports for Evaluation (mirroring your evaluation script) ---
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from skimage.metrics import structural_similarity as ssim

warnings.filterwarnings('ignore')

# --- CONFIGURATION ---
try:
    from google.colab import drive
    drive.mount('/content/drive')
    PROJECT_PATH = Path('/content/drive/My Drive/AR_Downscaling')
except:
    PROJECT_PATH = Path('.') # For local execution

DATA_DIR = PROJECT_PATH / 'final_dataset_multi_variable'
OUTPUT_DIR = PROJECT_PATH / 'publication_experiments' / 'operational_baseline_mos'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Define the target high resolution (should match Himawari data)
TARGET_SHAPE = (256, 256)

# --- DATA PREPARATION ---

def load_data(split='train'):
    """Loads and prepares the data for MOS training or testing."""
    print(f"Loading '{split}' data...")
    split_dir = DATA_DIR / split
    predictor_files = sorted(list(split_dir.glob('*_predictor.npy')))

    # Pre-allocate memory
    num_samples = len(predictor_files)
    # Assuming the first file tells us the coarse shape
    coarse_shape = np.load(predictor_files[0]).shape[1:]

    X_coarse = np.zeros((num_samples, 5, *coarse_shape), dtype=np.float32)
    Y_high_res = np.zeros((num_samples, *TARGET_SHAPE), dtype=np.float32)

    for i, pred_path in enumerate(tqdm(predictor_files, desc=f"Loading {split} files")):
        targ_path = Path(str(pred_path).replace('_predictor.npy', '_target.npy'))
        X_coarse[i] = np.load(pred_path)
        Y_high_res[i] = np.load(targ_path)

    return X_coarse, Y_high_res

def interpolate_predictors(X_coarse, target_shape):
    """Upscales coarse predictor variables to the high-resolution grid."""
    print("Interpolating predictors to high resolution...")
    num_samples, num_channels, _, _ = X_coarse.shape
    X_high_res = np.zeros((num_samples, num_channels, *target_shape), dtype=np.float32)

    zoom_factors = (1, target_shape[0] / X_coarse.shape[2], target_shape[1] / X_coarse.shape[3])

    for i in tqdm(range(num_samples), desc="Interpolating samples"):
        for c in range(num_channels):
            # Using bicubic interpolation (order=3)
            X_high_res[i, c] = zoom(X_coarse[i, c], zoom_factors[1:], order=3)

    return X_high_res

# --- MOS MODEL TRAINING ---

def train_mos_model(X_high_res_train, Y_high_res_train):
    """Trains a grid of linear regression models, one for each pixel."""
    print("Training MOS model (this may take a while)...")
    num_samples, num_channels, h, w = X_high_res_train.shape

    # Reshape data for scikit-learn
    # We want (pixel, sample, channel)
    X_train_reshaped = X_high_res_train.transpose(2, 3, 0, 1).reshape(h * w, num_samples, num_channels)
    Y_train_reshaped = Y_high_res_train.transpose(1, 2, 0).reshape(h * w, num_samples)

    # Create a grid to hold one trained model per pixel
    mos_model_grid = np.empty((h, w), dtype=object)

    for i in tqdm(range(h * w), desc="Training pixel-wise models"):
        row, col = i // w, i % w

        X_pixel = X_train_reshaped[i]
        Y_pixel = Y_train_reshaped[i]

        model = LinearRegression()
        model.fit(X_pixel, Y_pixel)

        mos_model_grid[row, col] = model

    # Save the trained model grid
    model_path = OUTPUT_DIR / 'mos_model_grid.joblib'
    joblib.dump(mos_model_grid, model_path)
    print(f"✅ MOS model saved to {model_path}")

    return mos_model_grid

# --- MOS PREDICTION AND EVALUATION ---

def predict_with_mos(mos_model_grid, X_high_res_test):
    """Generates predictions using the trained MOS model grid."""
    print("Generating predictions with MOS model...")
    num_samples, num_channels, h, w = X_high_res_test.shape

    # Reshape test data
    X_test_reshaped = X_high_res_test.transpose(2, 3, 0, 1).reshape(h * w, num_samples, num_channels)

    Y_pred_reshaped = np.zeros((h * w, num_samples), dtype=np.float32)

    for i in tqdm(range(h * w), desc="Predicting pixel-wise"):
        row, col = i // w, i % w
        model = mos_model_grid[row, col]

        Y_pred_reshaped[i] = model.predict(X_test_reshaped[i])

    # Reshape back to image format (samples, h, w)
    Y_pred = Y_pred_reshaped.reshape(h, w, num_samples).transpose(2, 0, 1)

    return Y_pred

def evaluate_predictions(Y_pred, Y_true):
    """Evaluates predictions using a comprehensive set of metrics."""
    print("Calculating final performance metrics...")
    all_metrics = []

    for i in tqdm(range(len(Y_pred)), desc="Evaluating samples"):
        pred_flat = Y_pred[i].flatten()
        true_flat = Y_true[i].flatten()

        # Calculate metrics
        rmse = np.sqrt(mean_squared_error(true_flat, pred_flat))
        mae = mean_absolute_error(true_flat, pred_flat)
        r2 = r2_score(true_flat, pred_flat)
        correlation = np.corrcoef(true_flat, pred_flat)[0, 1]

        # SSIM requires a data range
        data_range = Y_true[i].max() - Y_true[i].min()
        if data_range == 0:
            ssim_score = 1.0 if np.all(Y_pred[i] == Y_true[i]) else 0.0
        else:
            ssim_score = ssim(Y_true[i], Y_pred[i], data_range=data_range, win_size=7)

        all_metrics.append({
            'rmse': rmse, 'mae': mae, 'r2': r2,
            'correlation': correlation, 'ssim': ssim_score
        })

    # Calculate summary statistics
    results_df = pd.DataFrame(all_metrics)
    summary = results_df.agg(['mean', 'std', 'min', 'max']).transpose()

    print("\n--- MOS Baseline Performance Summary ---")
    print(summary)

    summary.to_csv(OUTPUT_DIR / 'mos_performance_summary.csv')
    print(f"\n✅ Evaluation results saved to {OUTPUT_DIR}")

    return summary

# --- MAIN EXECUTION SCRIPT ---

def main():
    # 1. Load Data
    X_train_coarse, Y_train_high_res = load_data('train')
    X_test_coarse, Y_test_high_res = load_data('test')

    # 2. Interpolate Predictors to Match Target Resolution
    X_train_interp = interpolate_predictors(X_train_coarse, TARGET_SHAPE)
    X_test_interp = interpolate_predictors(X_test_coarse, TARGET_SHAPE)

    # 3. Train the MOS model
    mos_model_grid = train_mos_model(X_train_interp, Y_train_high_res)

    # To load a pre-trained model instead:
    # print("Loading pre-trained MOS model...")
    # mos_model_grid = joblib.load(OUTPUT_DIR / 'mos_model_grid.joblib')

    # 4. Generate Predictions on the Test Set
    Y_pred_mos = predict_with_mos(mos_model_grid, X_test_interp)

    # 5. Evaluate the MOS Baseline
    evaluate_predictions(Y_pred_mos, Y_test_high_res)

if __name__ == "__main__":
    main()